In [1]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np

In [2]:
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)
df_teams.head(60)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


# Selección de partido
Se va a seleccionar el partido jugado el 13/12/2025 entre `okc` y `spurs`.

In [12]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog

In [35]:
partidos_temporada = leaguegamefinder.LeagueGameFinder(season_nullable="2025-26")
df_partidos_temporada = partidos_temporada.get_data_frames()[0]
df_partido_okc = df_partidos_temporada[((df_partidos_temporada['GAME_DATE'] == "2025-12-13") & 
                                    (df_partidos_temporada['TEAM_ABBREVIATION'] == 'OKC'))]
df_partido_okc

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
34,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,0.889,14,37,51,20,13,4,14,25,-2.0


In [36]:
game_id = df_partido_okc['GAME_ID'].iloc[0]
print(game_id)

0022501230


In [37]:
def extraer_sim(matchup):
    resultado = matchup[4]
    return resultado

def extraer_primero(matchup):
    resultado = matchup[0:3]
    return resultado

In [38]:
df_partidos_temporada['matchup_sim'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_sim)
df_partidos_temporada['matchup_pri'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_primero)
df_partidos_temporada['HA'] = np.where((df_partidos_temporada['TEAM_ABBREVIATION'] == df_partidos_temporada['matchup_pri'])&
    (df_partidos_temporada['matchup_sim'] == '@'), 'AWAY', np.where(
        (df_partidos_temporada['TEAM_ABBREVIATION'] != df_partidos_temporada['matchup_pri']) &
        (df_partidos_temporada['matchup_sim'] == 'v'),'AWAY', 'HOME'
    )
)
df_partidos_temporada.drop(columns=['matchup_sim', 'matchup_pri'], inplace=True)
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA
0,22025,1610612763,MEM,Memphis Grizzlies,0022500368,2025-12-17,MEM @ MIN,W,239,116,...,15,39,54,23,10,4,18,25,6.0,AWAY
1,22025,1610612750,MIN,Minnesota Timberwolves,0022500368,2025-12-17,MIN vs. MEM,L,239,110,...,16,36,52,26,11,8,18,25,-6.0,HOME
2,22025,1610612741,CHI,Chicago Bulls,0022500367,2025-12-17,CHI vs. CLE,W,240,127,...,6,37,43,35,12,3,15,17,16.0,HOME
3,22025,1610612739,CLE,Cleveland Cavaliers,0022500367,2025-12-17,CLE @ CHI,L,241,111,...,8,32,40,26,6,3,20,16,-16.0,AWAY
4,62025,1610612752,NYK,New York Knicks,0062500001,2025-12-16,SAS @ NYK,W,240,124,...,23,36,59,27,7,6,10,20,11.0,HOME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,10,40,50,29,9,5,16,31,22.0,AWAY
921,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,L,240,97,...,13,35,48,23,7,1,17,18,-10.2,AWAY
922,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,11,35,46,10,9,7,22,25,-22.0,HOME
923,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,20,38,58,19,13,2,17,26,15.0,HOME


In [39]:
df_partidos_temporada['GAME_ID'] = df_partidos_temporada['GAME_ID'].astype(str)
df_partidos_temporada['TEAM_ID'] = df_partidos_temporada['TEAM_ID'].astype(str)
df_partidos_temporada['GAME_TEAM_ID'] = df_partidos_temporada['GAME_ID']+'-'+df_partidos_temporada['TEAM_ID']
df_partidos_temporada = df_partidos_temporada.drop_duplicates(subset=['GAME_TEAM_ID'])
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID
0,22025,1610612763,MEM,Memphis Grizzlies,0022500368,2025-12-17,MEM @ MIN,W,239,116,...,39,54,23,10,4,18,25,6.0,AWAY,0022500368-1610612763
1,22025,1610612750,MIN,Minnesota Timberwolves,0022500368,2025-12-17,MIN vs. MEM,L,239,110,...,36,52,26,11,8,18,25,-6.0,HOME,0022500368-1610612750
2,22025,1610612741,CHI,Chicago Bulls,0022500367,2025-12-17,CHI vs. CLE,W,240,127,...,37,43,35,12,3,15,17,16.0,HOME,0022500367-1610612741
3,22025,1610612739,CLE,Cleveland Cavaliers,0022500367,2025-12-17,CLE @ CHI,L,241,111,...,32,40,26,6,3,20,16,-16.0,AWAY,0022500367-1610612739
4,62025,1610612752,NYK,New York Knicks,0062500001,2025-12-16,SAS @ NYK,W,240,124,...,36,59,27,7,6,10,20,11.0,HOME,0062500001-1610612752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,12025,1610612740,NOP,New Orleans Pelicans,0012500009,2025-10-03,NOP vs. MEL,W,240,107,...,33,46,20,11,8,16,22,10.0,HOME,0012500009-1610612740
920,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,40,50,29,9,5,16,31,22.0,AWAY,0012500001-1610612756
922,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,35,46,10,9,7,22,25,-22.0,HOME,0012500001-1610612747
923,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,38,58,19,13,2,17,26,15.0,HOME,0012500008-1610612752


In [40]:
equipos = list(df_teams['id'].astype(str))
equipos

['1610612737',
 '1610612738',
 '1610612739',
 '1610612740',
 '1610612741',
 '1610612742',
 '1610612743',
 '1610612744',
 '1610612745',
 '1610612746',
 '1610612747',
 '1610612748',
 '1610612749',
 '1610612750',
 '1610612751',
 '1610612752',
 '1610612753',
 '1610612754',
 '1610612755',
 '1610612756',
 '1610612757',
 '1610612758',
 '1610612759',
 '1610612760',
 '1610612761',
 '1610612762',
 '1610612763',
 '1610612764',
 '1610612765',
 '1610612766']

In [31]:
df_partidos_temporada = df_partidos_temporada.loc[
    df_partidos_temporada["TEAM_ID"].isin(equipos)
]
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID
0,22025,1610612739,CLE,Cleveland Cavaliers,0022500367,2025-12-17,CLE @ CHI,L,241,111,...,32,40,26,6,3,20,16,-16.0,AWAY,0022500367-1610612739
1,22025,1610612763,MEM,Memphis Grizzlies,0022500368,2025-12-17,MEM @ MIN,W,239,116,...,39,54,23,10,4,18,25,6.0,AWAY,0022500368-1610612763
2,22025,1610612741,CHI,Chicago Bulls,0022500367,2025-12-17,CHI vs. CLE,W,240,127,...,37,43,35,12,3,15,17,16.0,HOME,0022500367-1610612741
3,22025,1610612750,MIN,Minnesota Timberwolves,0022500368,2025-12-17,MIN vs. MEM,L,239,110,...,36,52,26,11,8,18,25,-6.0,HOME,0022500368-1610612750
4,62025,1610612759,SAS,San Antonio Spurs,0062500001,2025-12-16,SAS @ NYK,L,240,113,...,24,42,29,4,5,8,13,-11.0,AWAY,0062500001-1610612759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,12025,1610612740,NOP,New Orleans Pelicans,0012500009,2025-10-03,NOP vs. MEL,W,240,107,...,33,46,20,11,8,16,22,10.0,HOME,0012500009-1610612740
920,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,40,50,29,9,5,16,31,22.0,AWAY,0012500001-1610612756
921,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,35,46,10,9,7,22,25,-22.0,HOME,0012500001-1610612747
923,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,38,58,19,13,2,17,26,15.0,HOME,0012500008-1610612752


In [32]:
df_partidos_temporada.duplicated(subset=['GAME_TEAM_ID']).sum()

np.int64(0)

In [33]:
df_partidos_temporada[df_partidos_temporada.duplicated(subset=['GAME_TEAM_ID'], keep=False)]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA,GAME_TEAM_ID


In [15]:
partido = df_partidos_temporada[df_partidos_temporada['GAME_ID'] == game_id]
partido

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA
28,22025,1610612759,SAS,San Antonio Spurs,0022501230,2025-12-13,SAS @ OKC,W,241,111,...,15,37,52,21,9,6,19,23,2.0,AWAY
30,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,14,37,51,20,13,4,14,25,-2.0,HOME


In [16]:
from nba_api.stats.static import players

In [18]:
players = players.get_players()
df_players = pd.DataFrame(players)
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
5098,78650,Matt Zunic,Matt,Zunic,False
5099,1641783,Tristan da Silva,Tristan,da Silva,True
5100,1628427,Vlatko Čančar,Vlatko,Čančar,False
5101,1642365,Nikola Đurišić,Nikola,Đurišić,True


In [34]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import boxscoretraditionalv3

In [32]:
tiros = shotchartdetail.ShotChartDetail(
    team_id= 0,
    player_id= 0,
    season_type_all_star= 'Regular Season',
    season_nullable= '2025-26'
)

df_tiros =  tiros.get_data_frames()[0]
df_tiros.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022500001,10,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,11,26,...,Center(C),Less Than 8 ft.,1,0,13,1,1,20251021,OKC,HOU
1,Shot Chart Detail,0022500001,17,1631095,Jabari Smith Jr.,1610612745,Houston Rockets,1,11,2,...,Center(C),Less Than 8 ft.,0,-7,-5,1,1,20251021,OKC,HOU
2,Shot Chart Detail,0022500001,18,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,10,50,...,Center(C),Less Than 8 ft.,7,64,39,1,1,20251021,OKC,HOU
3,Shot Chart Detail,0022500001,27,201142,Kevin Durant,1610612745,Houston Rockets,1,10,7,...,Center(C),Less Than 8 ft.,0,0,-1,1,1,20251021,OKC,HOU
4,Shot Chart Detail,0022500001,29,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,9,52,...,Center(C),8-16 ft.,13,59,116,1,1,20251021,OKC,HOU


In [33]:
df_tiros.shape

(32426, 24)

In [37]:
boxscore = boxscoretraditionalv3.BoxScoreTraditionalV3(
    game_id= game_id
)
df_boxscore = boxscore.get_data_frames()[0]
df_boxscore.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631114,Jalen,Williams,J. Williams,...,1,6,7,4,4,0,2,2,17,-18.0
1,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631096,Chet,Holmgren,C. Holmgren,...,0,7,7,0,0,1,1,3,17,18.0
2,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628392,Isaiah,Hartenstein,I. Hartenstein,...,8,1,9,1,1,1,3,3,10,-7.0
3,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0,3,3,2,0,1,0,3,3,-10.0
4,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628983,Shai,Gilgeous-Alexander,S. Gilgeous-Alexander,...,0,4,4,5,0,1,5,4,29,2.0


In [ ]:
seasons = input('Ingrese las temporadas a cargar separadas por comas y sin espacios. Ejemplo(2024-25,2025-26)')

In [56]:
seasons = seasons.split(',')
seasons

['2024-25', '2025-26']

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
from src.etl.extract.games import fecth_games
from src.etl.extract.teams import fetch_teams

In [3]:
df_prueba = fecth_games(["2024-25","2025-26"])
df_prueba

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,SEASON_NUM,HA,GAME_TEAM_ID
0,42024,1610612760,OKC,Oklahoma City Thunder,0042400407,2025-06-22,OKC vs. IND,W,240,103,...,40,20,14,8,7,23,12.0,2024-25,HOME,0042400407-1610612760
1,42024,1610612754,IND,Indiana Pacers,0042400407,2025-06-22,IND @ OKC,L,240,91,...,45,17,6,4,21,24,-12.0,2024-25,AWAY,0042400407-1610612754
2,42024,1610612760,OKC,Oklahoma City Thunder,0042400406,2025-06-19,OKC @ IND,L,240,91,...,41,14,4,4,21,20,-17.0,2024-25,AWAY,0042400406-1610612760
3,42024,1610612754,IND,Indiana Pacers,0042400406,2025-06-19,IND vs. OKC,W,240,108,...,46,23,16,5,10,17,17.0,2024-25,HOME,0042400406-1610612754
4,42024,1610612760,OKC,Oklahoma City Thunder,0042400405,2025-06-16,OKC vs. IND,W,239,120,...,45,24,15,12,11,24,11.0,2024-25,HOME,0042400405-1610612760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,12025,1610612740,NOP,New Orleans Pelicans,0012500009,2025-10-03,NOP vs. MEL,W,240,107,...,46,20,11,8,16,22,10.0,2025-26,HOME,0012500009-1610612740
944,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,50,29,9,5,16,31,22.0,2025-26,AWAY,0012500001-1610612756
945,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,46,10,9,7,22,25,-22.0,2025-26,HOME,0012500001-1610612747
947,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,58,19,13,2,17,26,15.0,2025-26,HOME,0012500008-1610612752
